In [4]:
import cv2
import random
import os

from pathlib import Path
from types import SimpleNamespace
import shutil

CFG = SimpleNamespace()

# 01 - Extract frames from video

In [7]:
# Function to extract n random frames from a video file

def extract_random_frames(video_path, output_dir, n, size=(500, 500)):

    # Create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)
    
    # Open the video file
    video = cv2.VideoCapture(video_path)
    total_frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
    print(f"Total frames in video: {total_frames}")

    # Get n unique random frame indices
    frame_indices = sorted(random.sample(range(total_frames), n))
    
    saved = 0
    for idx in frame_indices:
        video.set(cv2.CAP_PROP_POS_FRAMES, idx)
        ret, frame = video.read()
        if ret:
            resized = cv2.resize(frame, size)
            filename = os.path.join(output_dir, f"frame_{idx}.jpg")
            cv2.imwrite(filename, resized)
            saved += 1
    video.release()
    print(f"Saved {saved} frames to {output_dir}")

In [ ]:
CFG.PATH_TO_VIDEO = 'videos/testvideo_1.mp4'
CFG.PATH_TO_FRAMES = 'frames'

extract_random_frames(
    video_path=CFG.PATH_TO_VIDEO,
    output_dir=CFG.PATH_TO_FRAMES,
    n=100,
    size=(640, 360),
    )

Total frames in video: 2731
Saved 100 frames to frames


# 02 - Reduce video size

In [15]:
import tqdm

# Reduce the size of the video file
def reduce_video_size(video_path, output_path, size=(640, 360)):
    # Open the video file
    video = cv2.VideoCapture(video_path)
    
    # Get the original video properties
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    fps = video.get(cv2.CAP_PROP_FPS)
    frame_count = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
    
    # Create VideoWriter object
    out = cv2.VideoWriter(output_path, fourcc, fps, size)

    with tqdm.tqdm(total=frame_count, desc="Processing frames") as pbar:
        while True:
            ret, frame = video.read()
            if not ret:
                break
            resized_frame = cv2.resize(frame, size)
            out.write(resized_frame)
            pbar.update(1)

    video.release()
    out.release()
    print(f"Video saved to {output_path}")

In [28]:
CFG.PATH_TO_VIDEO = 'videos/testvideo_1.mp4'
CFG.RED_SIZE = (640, 360)
CFG.PATH_TO_RED_VIDEO = Path(str(CFG.PATH_TO_VIDEO).replace('.mp4', f'_{CFG.RED_SIZE[0]}x{CFG.RED_SIZE[1]}.mp4'))

In [ ]:
reduce_video_size(
    video_path=CFG.PATH_TO_VIDEO,
    output_path=CFG.PATH_TO_RED_VIDEO,
    size=CFG.REDUCED_VIDEO_SIZE,
    )

Processing frames: 100%|██████████| 2731/2731 [00:07<00:00, 342.21it/s]

Video saved to videos\testvideo_1_640x360.mp4


# 03 - Split dataset

- Export annotated images in format 'YOLO with images'
- Unzip file and move the content to .\annotation_export
- Run the code below
    - The first cell to delete the existing dataset
    - The second cell to split the images in train/val and move them to the directories

In [7]:
# Delete the directory if it exists
CFG.DATASET_DIR_PARENT = Path().cwd() / 'dataset'

if CFG.DATASET_DIR_PARENT.exists() and CFG.DATASET_DIR_PARENT.is_dir():
    shutil.rmtree(CFG.DATASET_DIR_PARENT)

# Create the directories if they don't exist
dataset_dir_list = [
    CFG.DATASET_DIR_PARENT / 'images' / 'train',
    CFG.DATASET_DIR_PARENT / 'images' / 'val',
    CFG.DATASET_DIR_PARENT / 'labels' / 'train',
    CFG.DATASET_DIR_PARENT / 'labels' / 'val',
    ]

for path in dataset_dir_list:
    path.mkdir(parents=True)

In [9]:
path = Path().cwd() / 'annotation_export' / 'images'

# Find all the images in the folder
images = list(path.glob('*.jpg'))
print(f'{len(images)} images found in {path}')

# Set the number of images for training and validation
n_val = int(len(images) * 0.2)

# Take a random sample of 10 images for validation
val_images = random.sample(images, n_val)

for image in val_images:

    # Get the image name without the extension
    image_name = os.path.splitext(image.name)[0]

    # Get the corresponding label path
    label = path.parent / 'labels' / f'{image_name}.txt'

    # Create the new path for the image and label
    new_image_path = Path().cwd() / 'dataset' / 'images' / 'val' / f'{image_name}.jpg'
    new_label_path  = Path().cwd() / 'dataset' / 'labels' / 'val' / f'{image_name}.txt'

    # Move the files to the new path
    shutil.move(image, new_image_path)
    shutil.move(label, new_label_path)

print(f'{len(val_images)} images moved to validation set')

# Find all the images in the folder
images = list(path.glob('*.jpg'))

# Move the rest of the images to the training set
for image in images:
    # Get the image name without the extension
    image_name = os.path.splitext(image.name)[0]

    # Get the corresponding label path
    label = path.parent / 'labels' / f'{image_name}.txt'

    # Create the new path for the image and label
    new_image_path = Path().cwd() / 'dataset' / 'images' / 'train' / f'{image_name}.jpg'
    new_label_path  = Path().cwd() / 'dataset' / 'labels' / 'train' / f'{image_name}.txt'

    # Move the files to the new path
    shutil.move(image, new_image_path)
    shutil.move(label, new_label_path)

print(f'{len(images)} images moved to train set')

100 images found in c:\Users\thoma\Documents\GitHub\follow_target\annotation_export\images
20 images moved to validation set
80 images moved to train set
